In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json 
import cv2
import mediapipe as mp
import time

In [2]:
import sys
import os

In [3]:
%matplotlib inline

In [4]:
mp_holistic = mp.solutions.holistic # human pose, face landmarks, and hand tracking 
mp_drawing = mp.solutions.drawing_utils # To draw the keypoints

In [5]:
def mp_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # By deafult captured image isn't rgb
    image.flags.writeable = False # Can't edit during processing
    results = model.process(image)
    image.flags.writeable = True 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [6]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,      
    mp_drawing.DrawingSpec(
        color=(255,0,255),
        thickness=1,
        circle_radius=1
      ),
      mp_drawing.DrawingSpec(
        color=(0,255,255),
        thickness=1,
        circle_radius=1
      ))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [7]:
with open("WLASL_v0.3.json") as f:
    data = json.loads(f.read())

In [8]:
def get_video_ids(word):
    results = []
    for i in data:
        if(i['gloss'] == word):
            for j in i['instances']:
                results.append(j['video_id'])
    return results

In [9]:
# Extract the first 100 words
cnt = 0
words = []
for i in data:
    if(cnt==100):
        break
    words.append(i['gloss'])
    cnt+=1

In [10]:
# Finding the minimum frames in a video
min_frame_count = sys.maxsize
min_count = 1000000000
video_num = 1

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # NEW LOOP
    # Loop through actions
    for word in words:
        # print(f"Processing video no. {video_num}")
        videos = get_video_ids(word)
        # Loop through sequences aka videos
        count = 0
        for vid_id in videos:
            # Loop through video length 
            if(os.path.isfile(f"dataset/videos/{vid_id}.mp4")):
                count+= 1
                cap = cv2.VideoCapture(f"dataset/videos/{vid_id}.mp4")
                # no. of frames in a video
                length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
                if(length < min_frame_count):
                    min_frame_count = length
                cap.release()
                cv2.destroyAllWindows()
        if(count < min_count):
            min_count = count
        video_num+=1
print(min_frame_count)
print(min_count)

19
5


In [12]:
video_dir = './dataset/videos'

# Data map to store the video ids of the videos of the words present in the dataset
data_mp = dict()
for word in words:
    data_mp[word] = []

# Store the min frames count
min_frames_count = 1000000

for word in words:

    # Get video ids for of all the videos of a word
    video_ids = get_video_ids(word)

    for video_id in video_ids:
        videoPath = f"{video_dir}/{video_id}.mp4"
        video_reader = cv2.VideoCapture(videoPath)
        count  = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

        if (count!=0):
            data_mp[word].append(video_id)
            min_frames_count  = min(min_frames_count,count)

print("Minimum no. of frames: ",min_frames_count)

Minimum no. of frames:  19


In [11]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose, face, lh, rh])
    return np.concatenate([pose, lh, rh])

In [13]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # NEW LOOP
    # Loop through actions
    for word in words:
        videos = data_mp[word]
        cnt_word = 1
        # Loop through sequences aka videos
        for vid_id in videos:
            
            # Loop through video length aka sequence length
            cap = cv2.VideoCapture(f"dataset/videos/{vid_id}.mp4")
            count = 0
            cur_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            curr_skip_len = cur_len//min_frame_count
            fram_no = 0
            
            while cap.isOpened():
                # Read feed
                ret, frame = cap.read()
                if(not(ret)):
                    break
                # Make detections
                image, results = mp_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_landmarks(image, results)
                
                cv2.imshow("OpenCV Feed", image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                if(not(os.path.isdir(os.path.join("Data_without_face", word, vid_id)))):
                    os.makedirs(os.path.join("Data_without_face", word, vid_id))
                #file = open(os.path.join("Data", word, vid_id, str(count)+".npy"), 'x')
                #file.close()
                npy_path = os.path.join("Data_without_face", word, vid_id, str(count))
                

                if(not(os.path.isfile(npy_path))):
                    np.save(npy_path, keypoints)
                fram_no += curr_skip_len
                count+=1
                #print(count, min_frame_count)
                if(count+1 >= min_frame_count):
                    break
                currentPos = cap.get(cv2.CAP_PROP_POS_FRAMES)
                #print("Current", currentPos)
                #print("Supposed", fram_no)
                cap.set(cv2.CAP_PROP_POS_FRAMES, fram_no)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            #print("Count", count)
            #if(os.path.isdir(os.path.join("Data", word, vid_id))):
                #print(len(os.listdir(os.path.join("Data", word, vid_id))))
            cap.release()
            cv2.destroyAllWindows()

In [15]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # NEW LOOP
    # Loop through actions
    cnt_word = 1
    
    for word in words:
        videos = data_mp[word]
        # Loop through sequences aka videos
        
        print(f"Video no. {cnt_word} processing..")
        for vid_id in videos:
            
            # Loop through video length aka sequence length
            cap = cv2.VideoCapture(f"dataset/videos/{vid_id}.mp4")
            count = 0
            cur_len = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            curr_skip_len = cur_len//min_frame_count
            fram_no = 0
            
            while cap.isOpened():
                # Read feed
                ret, frame = cap.read()
                if(not(ret)):
                    break
                # Make detections
                image, results = mp_detection(frame, holistic)
                # print(results)

                # Draw landmarks
                draw_landmarks(image, results)
                
                cv2.imshow("OpenCV Feed", image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                if(not(os.path.isdir(os.path.join("Data_with_face", word, vid_id)))):
                    os.makedirs(os.path.join("Data_with_face", word, vid_id))
                #file = open(os.path.join("Data", word, vid_id, str(count)+".npy"), 'x')
                #file.close()
                npy_path = os.path.join("Data_with_face", word, vid_id, str(count))
                

                if(not(os.path.isfile(npy_path))):
                    np.save(npy_path, keypoints)
                fram_no += curr_skip_len
                count+=1
                #print(count, min_frame_count)
                if(count+1 >= min_frame_count):
                    break
                currentPos = cap.get(cv2.CAP_PROP_POS_FRAMES)
                #print("Current", currentPos)
                #print("Supposed", fram_no)
                cap.set(cv2.CAP_PROP_POS_FRAMES, fram_no)
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
            #print("Count", count)
            #if(os.path.isdir(os.path.join("Data", word, vid_id))):
                #print(len(os.listdir(os.path.join("Data", word, vid_id))))
            cap.release()
            cv2.destroyAllWindows()
            
        cnt_word +=1

Video no. 1 processing..
Video no. 2 processing..
Video no. 3 processing..
Video no. 4 processing..
Video no. 5 processing..
Video no. 6 processing..
Video no. 7 processing..
Video no. 8 processing..
Video no. 9 processing..
Video no. 10 processing..
Video no. 11 processing..
Video no. 12 processing..
Video no. 13 processing..
Video no. 14 processing..
Video no. 15 processing..
Video no. 16 processing..
Video no. 17 processing..
Video no. 18 processing..
Video no. 19 processing..
Video no. 20 processing..
Video no. 21 processing..
Video no. 22 processing..
Video no. 23 processing..
Video no. 24 processing..
Video no. 25 processing..
Video no. 26 processing..
Video no. 27 processing..
Video no. 28 processing..
Video no. 29 processing..
Video no. 30 processing..
Video no. 31 processing..
Video no. 32 processing..
Video no. 33 processing..
Video no. 34 processing..
Video no. 35 processing..
Video no. 36 processing..
Video no. 37 processing..
Video no. 38 processing..
Video no. 39 processi

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(words)}

In [13]:
label_map

{'book': 0,
 'drink': 1,
 'computer': 2,
 'before': 3,
 'chair': 4,
 'go': 5,
 'clothes': 6,
 'who': 7,
 'candy': 8,
 'cousin': 9,
 'deaf': 10,
 'fine': 11,
 'help': 12,
 'no': 13,
 'thin': 14,
 'walk': 15,
 'year': 16,
 'yes': 17,
 'all': 18,
 'black': 19,
 'cool': 20,
 'finish': 21,
 'hot': 22,
 'like': 23,
 'many': 24,
 'mother': 25,
 'now': 26,
 'orange': 27,
 'table': 28,
 'thanksgiving': 29,
 'what': 30,
 'woman': 31,
 'bed': 32,
 'blue': 33,
 'bowling': 34,
 'can': 35,
 'dog': 36,
 'family': 37,
 'fish': 38,
 'graduate': 39,
 'hat': 40,
 'hearing': 41,
 'kiss': 42,
 'language': 43,
 'later': 44,
 'man': 45,
 'shirt': 46,
 'study': 47,
 'tall': 48,
 'white': 49,
 'wrong': 50,
 'accident': 51,
 'apple': 52,
 'bird': 53,
 'change': 54,
 'color': 55,
 'corn': 56,
 'cow': 57,
 'dance': 58,
 'dark': 59,
 'doctor': 60,
 'eat': 61,
 'enjoy': 62,
 'forget': 63,
 'give': 64,
 'last': 65,
 'meet': 66,
 'pink': 67,
 'pizza': 68,
 'play': 69,
 'school': 70,
 'secretary': 71,
 'short': 72,
 '

In [14]:
import os
sequences, labels = [], []
cnt = 1
for word in words:
    print(f"Word No. {cnt} processing...")
    videos = get_video_ids(word)
    # Loop through sequences aka videos
    for vid_id in videos:
        window = []
        if(os.path.isdir(os.path.join("Data_without_face", word, vid_id))):
#             if(len(os.listdir(os.path.join("Data_without_face", word, vid_id))) + 1 <= 29):
#                 #print(vid_id)
            for frame_num in range(len(os.listdir(os.path.join("Data_without_face", word, vid_id)))):
                res = np.load(os.path.join("Data_without_face", word, vid_id, "{}.npy").format(frame_num))
                window.append(res)
                #print(len(res))
            sequences.append(window)
            #print(len(window))
            labels.append(label_map[word])

    # data augmentation        
    for vid_id in videos:
        window = []
        if(os.path.isdir(os.path.join("Data_without_face", word, vid_id))):
#             if(len(os.listdir(os.path.join("Data_without_face", word, vid_id))) + 1 <= 29):
#                 #print(vid_id)
            for frame_num in range(len(os.listdir(os.path.join("Data_without_face", word, vid_id)))):
                res = np.load(os.path.join("Data_without_face", word, vid_id, "{}.npy").format(frame_num))
                res/= 1.5
                window.append(res)
                 #print(len(res))
            sequences.append(window)
            #print(len(window))
            labels.append(label_map[word])
    
    cnt+=1

Word No. 1 processing...
Word No. 2 processing...
Word No. 3 processing...
Word No. 4 processing...
Word No. 5 processing...
Word No. 6 processing...
Word No. 7 processing...
Word No. 8 processing...
Word No. 9 processing...
Word No. 10 processing...
Word No. 11 processing...
Word No. 12 processing...
Word No. 13 processing...
Word No. 14 processing...
Word No. 15 processing...
Word No. 16 processing...
Word No. 17 processing...
Word No. 18 processing...
Word No. 19 processing...
Word No. 20 processing...
Word No. 21 processing...
Word No. 22 processing...
Word No. 23 processing...
Word No. 24 processing...
Word No. 25 processing...
Word No. 26 processing...
Word No. 27 processing...
Word No. 28 processing...
Word No. 29 processing...
Word No. 30 processing...
Word No. 31 processing...
Word No. 32 processing...
Word No. 33 processing...
Word No. 34 processing...
Word No. 35 processing...
Word No. 36 processing...
Word No. 37 processing...
Word No. 38 processing...
Word No. 39 processin

In [15]:
np.array(sequences).shape

(2026, 18, 258)

In [16]:
X= np.asarray(sequences)
X.shape

(2026, 18, 258)

In [28]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.15, random_state = 42)

In [31]:
words = np.array(words)

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, Bidirectional, Embedding
from tensorflow.keras.callbacks import TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(GRU(64, return_sequences=True, activation='relu', input_shape=(18,258)))
model.add(GRU(128, return_sequences=True, activation='relu'))
model.add(GRU(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(words.shape[0], activation='softmax'))


In [33]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [34]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
54/54 [==============================] - 10s 47ms/step - loss: 4.6015 - categorical_accuracy: 0.0122
Epoch 2/2000
54/54 [==============================] - 3s 48ms/step - loss: 4.4928 - categorical_accuracy: 0.0203
Epoch 3/2000
54/54 [==============================] - 2s 46ms/step - loss: 4.3081 - categorical_accuracy: 0.0232
Epoch 4/2000
54/54 [==============================] - 2s 45ms/step - loss: 4.2032 - categorical_accuracy: 0.0250
Epoch 5/2000
54/54 [==============================] - 2s 46ms/step - loss: 4.1567 - categorical_accuracy: 0.0290
Epoch 6/2000
54/54 [==============================] - 3s 47ms/step - loss: 4.1143 - categorical_accuracy: 0.0360
Epoch 7/2000
54/54 [==============================] - 3s 47ms/step - loss: 4.0975 - categorical_accuracy: 0.0279
Epoch 8/2000
54/54 [==============================] - 3s 49ms/step - loss: 4.0641 - categorical_accuracy: 0.0407
Epoch 9/2000
54/54 [==============================] - 2s 46ms/step - loss: 4.0683 - categorical

54/54 [==============================] - 3s 50ms/step - loss: 0.0625 - categorical_accuracy: 0.9826
Epoch 144/2000
54/54 [==============================] - 3s 51ms/step - loss: 0.3731 - categorical_accuracy: 0.9065
Epoch 145/2000
54/54 [==============================] - 3s 51ms/step - loss: 0.4439 - categorical_accuracy: 0.8763
Epoch 146/2000
54/54 [==============================] - 3s 52ms/step - loss: 0.8246 - categorical_accuracy: 0.7660
Epoch 147/2000
54/54 [==============================] - 3s 48ms/step - loss: 0.3589 - categorical_accuracy: 0.8972
Epoch 148/2000
54/54 [==============================] - 3s 50ms/step - loss: 0.1360 - categorical_accuracy: 0.9704
Epoch 149/2000
54/54 [==============================] - 3s 48ms/step - loss: 0.1017 - categorical_accuracy: 0.9779
Epoch 150/2000
54/54 [==============================] - 3s 50ms/step - loss: 0.2356 - categorical_accuracy: 0.9361
Epoch 151/2000
54/54 [==============================] - 3s 55ms/step - loss: 0.1828 - categoric

KeyboardInterrupt: 

In [35]:
yhat = model.predict(X_test)


In [36]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [37]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

multilabel_confusion_matrix(ytrue, yhat)

array([[[301,   0],
        [  0,   3]],

       [[298,   0],
        [  0,   6]],

       [[295,   4],
        [  0,   5]],

       [[301,   0],
        [  0,   3]],

       [[298,   2],
        [  0,   4]],

       [[301,   0],
        [  0,   3]],

       [[300,   1],
        [  0,   3]],

       [[296,   0],
        [  0,   8]],

       [[299,   0],
        [  0,   5]],

       [[302,   0],
        [  0,   2]],

       [[299,   0],
        [  0,   5]],

       [[297,   2],
        [  0,   5]],

       [[298,   0],
        [  0,   6]],

       [[300,   0],
        [  0,   4]],

       [[299,   2],
        [  0,   3]],

       [[301,   0],
        [  1,   2]],

       [[300,   0],
        [  0,   4]],

       [[303,   0],
        [  0,   1]],

       [[296,   0],
        [  2,   6]],

       [[302,   0],
        [  0,   2]],

       [[300,   0],
        [  0,   4]],

       [[298,   0],
        [  2,   4]],

       [[303,   0],
        [  0,   1]],

       [[298,   2],
        [  2, 

In [38]:
accuracy_score(ytrue, yhat)

0.8782894736842105

In [39]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
try:
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        count = 0
        while cap.isOpened():
            done = False
            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mp_detection(frame, holistic)
            print(results)

            # Draw landmarks
            draw_landmarks(image, results)

            # 2. Prediction logic
            keypoints = extract_keypoints(results)
    #         sequence.insert(0,keypoints)
    #         sequence = sequence[:30]
            sequence.append(keypoints)
            sequence = sequence[-23:]
            if len(sequence) == 23:
                res = model.predict(np.expand_dims(sequence, axis=0))[0]
                print(words[np.argmax(res)])
                predictions.append(np.argmax(res))


            #3. Viz logic
                if np.unique(predictions[-20:])[0]==np.argmax(res): 
                    if res[np.argmax(res)] > threshold: 

                        if len(sentence) > 0: 
                            if words[np.argmax(res)] != sentence[-1]:
                                sentence.append(words[np.argmax(res)])
                        else:
                            sentence.append(words[np.argmax(res)])

                if len(sentence) > 5: 
                    sentence = sentence[-5:]

                # Viz probabilities
                image = prob_viz(res, words, image, colors)
                    
            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
except Exception as e:
    print(e)
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti